In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
l = .8
M = 1.
m =.1
g = 9.8
dT = 0.02

## Pure Python

In [3]:
import math

def calc_rhe_pure_python(q):
    return [q[2], 
            q[3], 
             (g*m*math.sin(2*q[1])/2 + l*m*q[3]**2*math.sin(q[1]))/(M + m*math.sin(q[1])**2),
            -(g*(M + m)*math.sin(q[1]) + (l*m*q[3]**2*math.sin(q[1]))*math.cos(q[1]))/(l*(M + m*math.sin(q[1])**2))]

def RK4_pure_python(q, dt):
    k1 = calc_rhe_pure_python(q)
    k2 = calc_rhe_pure_python([e+d*dt/2 for e, d in zip(q, k1)])
    k3 = calc_rhe_pure_python([e+d*dt/2 for e, d in zip(q, k2)])
    k4 = calc_rhe_pure_python([e+d*dt/2 for e, d in zip(q, k3)])
    return [x+dt*(k1x/6+k2x/3+k3x/3+k4x/6) for x, k1x, k2x, k3x, k4x in zip(q, k1, k2, k3, k4)]

In [4]:
x0 = [0., -math.pi+0.1, 0., 0.]

print("1. Pure Python\n\t:", end='')
%timeit x1 = RK4_pure_python(x0, dT)

1. Pure Python
	:12.5 µs ± 283 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Numpy

In [5]:
def calc_rhe_numpy(q):
    return np.array([q[2], 
            q[3], 
             (g*m*np.sin(2*q[1])/2 + l*m*q[3]**2*np.sin(q[1]))/(M + m*np.sin(q[1])**2),
            -(g*(M + m)*np.sin(q[1]) + (l*m*q[3]**2*np.sin(q[1]))*np.cos(q[1]))/(l*(M + m*np.sin(q[1])**2))])

def RK4_numpy(q, dt):
    k1 = calc_rhe_numpy(q)
    k2 = calc_rhe_numpy(q+k1*dt/2)
    k3 = calc_rhe_numpy(q+k2*dt/2)
    k4 = calc_rhe_numpy(q+k3*dt/2)
    return q+dt*(k1/6+k2/3+k3/3+k4/6)

In [6]:
x0 = np.array([0., -math.pi+0.1, 0., 0.])

print("2. Numpy\n\t:", end='')
%timeit x1 = RK4_numpy(x0, dT)

2. Numpy
	:66.2 µs ± 1.61 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Numba - Pure python

In [7]:
from numba import jit

@jit(nopython=True)
def calc_rhe_pure_python_numba(q):
    return [q[2], 
            q[3], 
             (g*m*math.sin(2*q[1])/2 + l*m*q[3]**2*math.sin(q[1]))/(M + m*math.sin(q[1])**2),
            -(g*(M + m)*math.sin(q[1]) + (l*m*q[3]**2*math.sin(q[1]))*math.cos(q[1]))/(l*(M + m*math.sin(q[1])**2))]

@jit(nopython=True)
def RK4_pure_python_numba(q, dt):
    k1 = calc_rhe_pure_python_numba(q)
    k2 = calc_rhe_pure_python_numba([e+d*dt/2 for e, d in zip(q, k1)])
    k3 = calc_rhe_pure_python_numba([e+d*dt/2 for e, d in zip(q, k2)])
    k4 = calc_rhe_pure_python_numba([e+d*dt/2 for e, d in zip(q, k3)])
    return [x+dt*(k1x/6+k2x/3+k3x/3+k4x/6) for x, k1x, k2x, k3x, k4x in zip(q, k1, k2, k3, k4)]

In [8]:
x0 = [0., -math.pi+0.1, 0., 0.]

# initial call for JIT compiler
x1 = RK4_pure_python_numba(x0, dT)

print("3. Numba - Pure Python\n\t:", end='')
%timeit x1 = RK4_pure_python_numba(x0, dT)

3. Numba - Pure Python
	:8.82 µs ± 72.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Numba - Numpy

In [9]:
from numba import jit

@jit(nopython=True)
def calc_rhe_numpy_numba(q):
    return np.array([q[2], 
            q[3], 
             (g*m*np.sin(2*q[1])/2 + l*m*q[3]**2*np.sin(q[1]))/(M + m*np.sin(q[1])**2),
            -(g*(M + m)*np.sin(q[1]) + (l*m*q[3]**2*np.sin(q[1]))*np.cos(q[1]))/(l*(M + m*np.sin(q[1])**2))])

@jit(nopython=True)
def RK4_numpy_numba(q, dt):
    k1 = calc_rhe_numpy_numba(q)
    k2 = calc_rhe_numpy_numba(q+k1*dt/2)
    k3 = calc_rhe_numpy_numba(q+k2*dt/2)
    k4 = calc_rhe_numpy_numba(q+k3*dt/2)
    return q+dt*(k1/6+k2/3+k3/3+k4/6)

In [10]:
x0 = np.array([0., -math.pi+0.1, 0., 0.])

# initial call for JIT compiler
x1 = RK4_numpy_numba(x0, dT)

print("4. Numba Numpy\n\t:", end='')
%timeit x1 = RK4_numpy_numba(x0, dT)

4. Numba Numpy
	:1.46 µs ± 22.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Cython

In [11]:
%load_ext Cython

In [12]:
%%cython
import numpy as np
cimport numpy as np
cimport cython

ctypedef double DTYPE_t

cdef double l = .8
cdef double M = 1.
cdef double m =.1
cdef double g = 9.8
cdef double dT = 0.02

@cython.boundscheck(False)
@cython.wraparound(False)
cdef calc_rhe_numpy_cython(np.ndarray[DTYPE_t, ndim=1] q):
    return np.array([q[2], 
            q[3], 
             (g*m*np.sin(2*q[1])/2 + l*m*q[3]**2*np.sin(q[1]))/(M + m*np.sin(q[1])**2),
            -(g*(M + m)*np.sin(q[1]) + (l*m*q[3]**2*np.sin(q[1]))*np.cos(q[1]))/(l*(M + m*np.sin(q[1])**2))])

@cython.boundscheck(False)
@cython.wraparound(False)
def RK4_numpy_cython(np.ndarray[DTYPE_t, ndim=1] q, double dt):
    cdef np.ndarray[DTYPE_t, ndim=1] k1
    cdef np.ndarray[DTYPE_t, ndim=1] k2
    cdef np.ndarray[DTYPE_t, ndim=1] k3
    cdef np.ndarray[DTYPE_t, ndim=1] k4

    k1 = calc_rhe_numpy_cython(q)
    k2 = calc_rhe_numpy_cython(q+k1*dt/2)
    k3 = calc_rhe_numpy_cython(q+k2*dt/2)
    k4 = calc_rhe_numpy_cython(q+k3*dt/2)
    return q+dt*(k1/6+k2/3+k3/3+k4/6)

In [13]:
x0 = np.array([0., -math.pi+0.1, 0., 0.])

print("6. Cython - Numpy\n\t:", end='')
%timeit x1 = RK4_numpy_cython(x0, dT)

6. Cython - Numpy
	:52 µs ± 693 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Sympy

In [14]:
 import sympy as sy
 def gen_rhe_sympy():
        q  = sy.symbols('q:{0}'.format(4))
        qd = q[2:4]
        
        I = sy.Matrix([[1, 0, 0, 0], 
                      [0, 1, 0, 0], 
                      [0, 0, M + m, l*m*sy.cos(q[1])], 
                      [0, 0, l*m*sy.cos(q[1]), l**2*m]])
        f = sy.Matrix([
                       qd[0], 
                       qd[1],
                       l*m*sy.sin(q[1])*qd[1]**2,
                      -g*l*m*sy.sin(q[1])])
        return sy.lambdify([q], sy.simplify(I.inv()*f))

calc_rhe_sympy = gen_rhe_sympy()

def RK4_sympy(q, dt):
    k1 = calc_rhe_sympy(q)
    k2 = calc_rhe_sympy(q+k1*dt/2)
    k3 = calc_rhe_sympy(q+k2*dt/2)
    k4 = calc_rhe_sympy(q+k3*dt/2)
    return q+dt*(k1/6+k2/3+k3/3+k4/6)

In [15]:
x0 = np.array([0., -math.pi+0.1, 0., 0.])

print("7. Sympy - Numpy\n\t:", end='')
%timeit x1 = RK4_sympy(x0, dT)

7. Sympy - Numpy
	:97.3 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Eigen - Swig
```
python setup.py build_ext --inplace
```

In [16]:
from swig_eigen.swig_eigen_mod import RK4_eigen

In [17]:
x0 = np.array([0., -math.pi+0.1, 0., 0.])

print("8. Swig -Eigen\n\t:", end='')
%timeit x1 = RK4_eigen(x0, dT)

8. Swig -Eigen
	:1.89 µs ± 15.4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
